In [ ]:
using Statistics, Random, DataFrames, CSV,Plots,StatsBase,Distributions,LinearAlgebra

In [ ]:
data_path = "/Users/ziguiwang/Documents/Animal/Functional_Annoation/FAANG_simulation/largedata/GALLO_MSU_pig/"
result_path = "/Users/ziguiwang/Documents/Animal/Functional_Annoation/FAANG_simulation/reports/fixed_marker/"
raw_genotype = CSV.read(data_path*"pig_genotype.csv", DataFrame);

### Read genotype, map, and annotation data

In [ ]:
new_genotype      = raw_genotype[!,2:end]; # remove the ID column
ID                = raw_genotype[!,1];     # get the ID column

annotation_map    = CSV.read(data_path*"pig_FA.csv", DataFrame);   # read annoation information for each SNP
map_data          = CSV.read(data_path*"pig_map.csv", DataFrame);  # read map data for MSU pig data

number_snp        = size(map_data,1);      # get number of SNPs
number_individual = size(raw_genotype,1);  # get number of individuals

### Find the index for SNPs in each annotation class

In [ ]:
Random.seed!(345)

# get the snp index that belong to each genome annotation
psudogene_index            = annotation_map[!,2] .== "pseudogene"
protein_coding_index       = annotation_map[!,2] .== "protein_coding"
processed_pseudogene_index = annotation_map[!,2] .== "processed_pseudogene"
snoRNA_index               = annotation_map[!,2] .== "snoRNA"
snRNA_index                = annotation_map[!,2] .== "snRNA"
rRNA_index                 = annotation_map[!,2] .== "rRNA"
miRNA_index                = annotation_map[!,2] .== "miRNA"
miscRNA_index              = annotation_map[!,2] .== "misc_RNA"

# get the snp name that belong to each genome annotation
psudogene_snp              = annotation_map[psudogene_index,1];
protein_coding_snp         = annotation_map[protein_coding_index,1];
processed_pseudogene_snp   = annotation_map[processed_pseudogene_index,1];
snoRNA_snp                 = annotation_map[snoRNA_index,1];
snRNA_snp                  = annotation_map[snRNA_index,1];
rRNA_snp                   = annotation_map[rRNA_index,1];
miRNA_snp                  = annotation_map[miRNA_index,1];
miscRNA_snp                = annotation_map[miscRNA_index,1];


# find the index of the SNPs of each annotation class from the map data
psudogene_snp_index        = findall(x->x in psudogene_snp , map_data[!,:snp]);
protein_coding_snp_index   = findall(x->x in protein_coding_snp , map_data[!,:snp]);
processed_pseudogene_snp_index   = findall(x->x in processed_pseudogene_snp , map_data[!,:snp]);
snoRNA_snp_index                 = findall(x->x in snoRNA_snp , map_data[!,:snp]);

snRNA_snp_index                  = findall(x->x in snRNA_snp , map_data[!,:snp]);
rRNA_snp_index                   = findall(x->x in rRNA_snp , map_data[!,:snp]);
miRNA_snp_index                  = findall(x->x in miRNA_snp , map_data[!,:snp]);
miscRNA_snp_index                = findall(x->x in miscRNA_snp , map_data[!,:snp]);

RNA_snp_index                    = vcat(snoRNA_snp_index,snRNA_snp_index,rRNA_snp_index,miRNA_snp_index,miscRNA_snp_index );
intergenic_snp_index             = findall(!(x->x in annotation_map[!,:snp]) , map_data[!,:snp])

# merge all non protein coding index together
non_protein_coding_snp_index         = vcat(RNA_snp_index,psudogene_snp_index,processed_pseudogene_snp_index,intergenic_snp_index   );

### set up simulation function

In [ ]:
function simulate_bv_multiple(new_genotype ,n,simu_qtl_index,id) 

    
    number_qtl             = size(simu_qtl_index,1)
    
    simu_qtl_genotype      = Matrix(new_genotype[1:n,simu_qtl_index ]);
    
    sigma_g = 1.0 # assume the genetic variance equals to 1
    
    
    # simulate qtl effect for two traits
    simu_qtl_effect_trait1 = randn(number_qtl )* sigma_g
    simu_qtl_effect_trait2 = randn(number_qtl )* sigma_g
   
    simu_qtl_effect        = [simu_qtl_effect_trait1 simu_qtl_effect_trait2]
    
    # set the genetic covariance
    covG = 0.2  
    G    = [1 covG                        # genetic covariance matrix
            covG 1 ]
    
    GU   = cholesky(G).U
    simu_qtl_effect = simu_qtl_effect*GU;
    

    # simulate breeding value
    M   =  Matrix(new_genotype[:,1:end])
    BV  =  M[:,simu_qtl_index]*simu_qtl_effect
    
    #std_matrix = Diagonal(vec(std(BV,dims = 1)))    # set varg = 1 for convenience
    #BV         = BV*inv(std_matrix)
    
    

    phenotype = DataFrame(ID = id, b1 = BV[:,1],b2 = BV[:,2])

    return phenotype
    
end


function simulate_phenotype_multiple( group1_bv ,group2_bv ,n, id,h2_trait1,h2_trait2)
    
    # generate the breeding value matrix
    
    BV      = Matrix(group1_bv[:,2:end]) + Matrix(group2_bv[:,2:end]) 
    
    
    # standardized the BV
    
    std_matrix = Diagonal(vec(std(BV,dims = 1)))    # set varg = 1 for convenience
    BV         = BV*inv(std_matrix)

    #println("varaince for trait 1 BV: ",var(BV[:,1]))
    #println("varaince for trait 2 BV: ",var(BV[:,2]))
    
    println("covariance for BV is : ",cov(BV[:,1],BV[:,2]))
    # simulate the residual 
    residual        = randn(n,2)
    varg_trait1     = var(BV[:,1])
    varg_trait2     = var(BV[:,2]) 
    vare_trait1     = (1 - h2_trait1)/h2_trait1 *varg_trait1       # residual variance
    vare_trait2     = (1 - h2_trait2)/h2_trait2 *varg_trait2       # residual variance
    
    
    #println("residual variance trait 1 is : ",vare_trait1)
    #println("residual variance trait 2 is : ",vare_trait2)
    
    covR            = 0.0      
    R               = [vare_trait1  covR                 # residual covariance matrix
                       covR  vare_trait2]

    RU       = cholesky(R).U
    residual =  residual*RU;

    # simulate phenotypes

    pheno1  = BV[:,1] + residual[:,1];   
    
    pheno2  = BV[:,2] + residual[:,2];
    
    phenotype = DataFrame(ID = id, y1 = pheno1,y2 = pheno2, b1 = BV[:,1],b2 = BV[:,2])

    return phenotype
    
end


#### function test

In [ ]:
group1_bv = simulate_bv_multiple(new_genotype ,number_individual,protein_coding_QTL_index,ID); # get the simulated breeding value in protein group
group2_bv = simulate_bv_multiple(new_genotype ,number_individual,non_protein_coding_QTL_index,ID);# get the simulated breeding value in non protein group
MSU_simu_pheno = simulate_phenotype_multiple( group1_bv ,group2_bv ,number_individual, ID, 0.6,0.8);

### Start the simulation

In [ ]:
Random.seed!(345)

# simulate 2000 QTLs from protein coding SNPs
protein_coding_QTL_index         = protein_coding_snp_index[sample(1:size(protein_coding_snp_index,1  ),2000,replace = false)];
# simulate 100 QTLs from non protein coding SNPs
non_protein_coding_QTL_index      = non_protein_coding_snp_index[sample(1:size(non_protein_coding_snp_index,1  ),100,replace = false)];
# combine all QTLs index together
candidate_QTL_index              = unique(vcat(protein_coding_QTL_index, non_protein_coding_QTL_index  ));

In [ ]:
pheno_index = 1

for i in 1:30 # simulate 30 datasets
    group1_bv = simulate_bv_multiple(new_genotype ,number_individual,protein_coding_QTL_index,ID); # get the simulated breeding value in protein group
    group2_bv = simulate_bv_multiple(new_genotype ,number_individual,non_protein_coding_QTL_index,ID);# get the simulated breeding value in non protein group
    MSU_simu_pheno = simulate_phenotype_multiple( group1_bv ,group2_bv ,number_individual, ID, 0.6,0.8); # combine two breeding value and simualte the phenotype
    
    #CSV.write( data_path*"multi-trait-pheno/different_h2_MSU_multi_simu_pheno_"*string(i)*".csv" , MSU_simu_pheno)
end

In [ ]:
MSU_simu_pheno = CSV.read(data_path*"multi-trait-pheno/different_h2_MSU_multi_simu_pheno_3.csv", DataFrame);
cov(MSU_simu_pheno[!,:b1],MSU_simu_pheno[!,:b2])

### Check the marker variance and genetic variance of trait 1 (y1)

In [ ]:
group1_bv = simulate_bv_multiple(new_genotype ,number_individual,protein_coding_QTL_index,ID); # get the simulated breeding value in protein group
group2_bv = simulate_bv_multiple(new_genotype ,number_individual,non_protein_coding_QTL_index,ID);# get the simulated breeding value in non protein group
MSU_simu_pheno = simulate_phenotype_multiple( group1_bv ,group2_bv ,number_individual, ID, 0.6,0.8);

In [ ]:
# define some prior parameter

geno = new_genotype
pi = 0.99
h2 = 0.5
number_markers = size(geno,2) - 1 # compute the number of marker, -1 since the ID column

# compute the genetic variance from the phenotypic variance
phenotypic_var = var(MSU_simu_pheno[!,:y1])
genetic_var = phenotypic_var*h2
markerMeans = mean(Matrix(geno[!,2:end]), dims=1)
p = markerMeans/2.0
mean2pq = mean(2*transpose(p) .* (1 .- transpose(p) ))
var_effects = genetic_var/(number_markers *(1-pi)* mean2pq )